# Title of the Notebook
- Date: 
- Purpose:

## Notebook Setup
- Business question: 
- Expected outcomes:
- Assumptions or constraints: 

## Notebook environment setup

In [1]:
import os
import sys
from datetime import datetime

def set_working_directory(path: str) -> None:
    """
    Set the working directory and add it to sys.path for module imports.

    Args:
        path (str): The path to set as the working directory.

    Raises:
        FileNotFoundError: If the specified path does not exist.
    """
    if not os.path.isdir(path):
        raise FileNotFoundError(f"Directory '{path}' does not exist.")
    os.chdir(path)
    if path not in sys.path:
        sys.path.insert(0, path)

# Example usage:
set_working_directory("C:\\Users\\izzaz\\Documents\\2 Areas\\GitHub\\ai-tools")

src_path = os.path.join(os.getcwd(), "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

### Visualization params and styling

In [ ]:
import matplotlib.pyplot as plt
plt.style.use(['seaborn-v0_8-notebook', 'seaborn-v0_8-white'])

## OLS Data
### Get Data (if needed)

In [ ]:
from src.core.ga4 import OLS
ols = OLS()
df = ols.get_ols_dataframe("2024-01-01", "yesterday")
try:
    df.to_csv("data/raw/ols.csv", index=True)
except Exception as e:
    print(f"Error saving CSV: {e}")


### Load Data

In [2]:
import pandas as pd
# If re-accessing this notebook, read from CSV instead of reprocessing
df = pd.read_csv("data/raw/ols.csv", index_col=0, parse_dates=True)

### Reshape data to weekly

In [3]:
def process_ols_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the OLS data for weekly aggregation.
    """
    return (df
    .reset_index()
    .assign(date=pd.to_datetime(df.index, format="%Y%m%d"))
    .set_index("date")
    .resample("W-SUN")
    .sum()
    .assign(week=lambda x: x.index.isocalendar().week,
            year=lambda x: x.index.isocalendar().year)
 )

processed_df = process_ols_data(df)
processed_df.tail(10)

,traffic,view_item,type_addr,add_contact,add_addr,add_date,checkout,purchase,week,year
date,,,,,,,,,,
2025-02-23,118140,4923,1528,836,930,517,466,351,8,2025
2025-03-02,158300,6323,1876,1210,1328,844,493,361,9,2025
2025-03-09,140929,6260,1816,600,1174,641,507,385,10,2025
2025-03-16,131538,5560,1555,240,957,531,501,366,11,2025
2025-03-23,106207,5252,1337,602,783,463,419,319,12,2025
2025-03-30,133238,5131,1332,909,764,480,506,350,13,2025
2025-04-06,176808,6573,1515,884,736,469,394,300,14,2025
2025-04-13,213996,6035,1456,989,817,471,483,355,15,2025
2025-04-20,151879,5269,1388,1015,856,546,507,340,16,2025


### Data Exploration

Reshape data to json

In [4]:
json_data = processed_df.reset_index().to_dict(orient="records")

Initial EDA

In [5]:
from src.core.openai_client import OpenAIClient

client = OpenAIClient()
prompt = f"""
        You are a data analyst. I will provide you with a JSON object containing weekly data.
        Your task is to analyze the data and provide insights, focusing on the last week in the dataset.

        Analysis steps:
        - Provide comparisons with previous weeks.
        - Provide comparisons with previous year.
        - Highlight key trends, ascending or descending.
        - Perform funnel and conversion rate analysis.


        The dataset contains the following fields:
        - date: The date of the week (YYYY-MM-DD format)
        - week: The week number of the year
        - year: The year of the week
        - traffic: Total traffic in users for the week
        - view_item: Total users who started the conversion funnel process
        - type_addr: Total users who typed into the address bar
        - add_contact: Total users who added their contact, also known as "leads drop"
        - add_addr: Total users who are within our service coverage area and able to add their address
        - add_date: Total users who added their preferred service installation date
        - checkout: Total users who began the checkout process
        - purchase: Total users who completed the sign up process

        Context: we are a telecommunications company that provides home fibre internet service. We do not have service in all areas, so we have a service coverage area. The website is a key channel for acquiring new customers.

        Dataset: {json_data}
"""
try:
    response = client.get_completion(prompt)
    print(response["tokens"])
    print(response["cost"])
except Exception as e:
    print(f"Error getting response: {e}")

{'input': 6387, 'output': 1585, 'total': 7972}
{'cost in USD ($)': 0.005091}


In [6]:
# Output the response to a markdown file
if response:
    try:
        with open(f"reports/{datetime.now().strftime("%Y%m%d")}_ols_analysis.md", "w") as f:
            f.write(response["text"])
        print("Analysis saved to reports folder.")
    except Exception as e:
        print(f"Error writing to file: {e}")

Analysis saved to reports folder.


### Conversion Rate Analysis

In [7]:
prompt = """
    Perform conversion rate analysis on the website data.

    Calculate the conversion rates for the past week and compare it to previous weeks' distributions. Include what percentile of value past week falls into. Flag all conversion rates that are below the 25th percentile, and highlight any significant changes in conversion rates.

    Only output the results and insights, nothing else.

    Example:

    {
        "week": 17,
        "year": 2025,
        "conversion_rates": {
        "traffic_to_view_item": 0.0344,
        "view_item_to_type_addr": 0.253,
        "type_addr_to_add_contact": 0.644,
        "add_contact_to_add_addr": 0.847,
        "add_addr_to_add_date": 0.637,
        "add_date_to_checkout": 0.927,
        "checkout_to_purchase": 0.761,
        "traffic_to_purchase": 0.00214
    },
        "percentiles_vs_previous_weeks": {
        "traffic_to_view_item": 30,
        "view_item_to_type_addr": 10,
        "type_addr_to_add_contact": 60,
        "add_contact_to_add_addr": 50,
        "add_addr_to_add_date": 45,
        "add_date_to_checkout": 70,
        "checkout_to_purchase": 50,
        "traffic_to_purchase": 35
    },
    "flags": {
    "below_25th_percentile": [
        "view_item_to_type_addr"
        ]
    },
        "significant_changes": [
        "View_item to type_addr conversion rate is significantly lower than typical weeks and is flagged below the 25th percentile at 10th percentile.",
        "Traffic to view_item rate is slightly below median but above 25th percentile, no flag.",
        "All other conversion rates fall within normal or better-than-normal ranges."
    ],
    "insights": [
        "The largest area of concern is the low conversion rate from view_item to type_addr, highlighting a critical funnel bottleneck likely affecting overall lead generation.",
        "Despite strong traffic, fewer users proceed to type their address than in most previous weeks, which may be due to UX issues or service availability perceptions.",
        "Maintaining or improving conversion rates at downstream steps is positive but cannot compensate for the significant drop-off earlier in the funnel.",
        "Focus efforts on optimizing the user experience and clarity at the address input stage to improve funnel throughput and overall conversions."
    ]
    }
"""
followup_response = client.get_completion(prompt, keep_history=True)
print(followup_response["tokens"])
print(followup_response["cost"])

{'input': 525, 'output': 491, 'total': 1016}
{'cost in USD ($)': 0.000996}


In [8]:
# Output the response to a markdown file
if response:
    try:
        with open(f"reports/{datetime.now().strftime("%Y%m%d")}_funnel_analysis.md", "w") as f:
            f.write(response["text"])
        print("Analysis saved to reports folder.")
    except Exception as e:
        print(f"Error writing to file: {e}")

Analysis saved to reports folder.


## Sales

In [ ]:
from src.core.mongodb import Mongo

client = Mongo("deep-diver-v2", "boreport")
sales_df = client.get_data()
sales_df.head(5)

In [ ]:
online = (sales_df
 .loc[sales_df[" Channel"].isin(["ONLINE"])]
 .dropna(subset=["Probability 90% Date"])
 .assign(date=pd.to_datetime(sales_df["Probability 90% Date"], format="%Y-%m-%d %H:%M:%S"))
 .set_index("date")
 .resample("W-SUN")
 ["Funnel SO No"]
 .count()
 )

online.tail(20)